In [4]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 445.2 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/250.9 kB 445.2 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/250.9 kB 445.2 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 179.6 kB/s eta 0:00:02
   ------------ -------------------------- 81.9/250.9 kB 306.3 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 327.7 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 327.7 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.9 kB 273.1 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 316.2 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.9 kB 351.5 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.9 kB 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import requests
import pandas as pd
import threading

def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return (username, password)

def obter_landing_pages(microsite_id, page=1):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/assets/landingPages?search=micrositeId={microsite_id}&page={page}"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()  

    return response.json()

def obter_landing_page_detalhes(landing_page_id):
    url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/landingPage/{landing_page_id}"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()
    return response.json()

def obter_dados_landing_page(landing_page_id):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/data/report/5?assetId={landing_page_id}&startAt=1715620884"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()
    return response.json()

def processar_landing_page(landing_page, dados):
    landing_page_id = landing_page["id"]
    landing_page_name = landing_page["name"]

    print(f"Processando landing page {landing_page_name}...")

    # Obter detalhes da landing page para acessar os forms
    landing_page_detalhes = obter_landing_page_detalhes(landing_page_id)
    forms = landing_page_detalhes.get("forms", [])

    # Consultar dados da landing page
    dados_landing_page = obter_dados_landing_page(landing_page_id)

    # Calcular o retorno
    total = dados_landing_page["total"]
    retorno = "Y" if total > 0 else "N"

    if forms:
        for form in forms:
            form_name = form["name"]
            form_id = form["id"]
            redirect_name = ""

            # Procurar por FormStepRedirectToWebPage em processingSteps
            for step in form.get("processingSteps", []):
                if step["type"] == "FormStepRedirectToWebPage":
                    if "pageUrl" in step:
                        redirect_name = step["pageUrl"]["constantValue"]
                    elif "landingPageId" in step:
                        redirect_id = step["landingPageId"]["constantValue"]
                        # Obter detalhes do landing page de redirecionamento
                        redirect_details = obter_landing_page_detalhes(redirect_id)
                        redirect_name = redirect_details["name"]
                    else:
                        redirect_name = ""
                    break

            dados.append({
                "landingpage name": landing_page_name,
                "landingPage ID": landing_page_id,
                "Form Name": form_name,
                "Form ID": form_id,
                "Redirect Name": redirect_name,
                "Return": retorno  # Adicionando a coluna Return
            })
    else:
        # Se não houver forms, adiciona os campos em branco
        dados.append({
            "landingpage name": landing_page_name,
            "landingPage ID": landing_page_id,
            "Form Name": "",
            "Form ID": "",
            "Redirect Name": "",
            "Return": retorno  # Adicionando a coluna Return
        })

def processar_landing_pages_threads(microsite_id):
    # Obter a primeira página para descobrir o total e o pageSize
    landing_pages = obter_landing_pages(microsite_id)
    total = landing_pages["total"]
    pageSize = landing_pages["pageSize"]
    num_pages = (total // pageSize) + (1 if total % pageSize else 0)

    dados = []
    threads = []
    
    for page in range(1, num_pages + 1):
        landing_pages = obter_landing_pages(microsite_id, page)
        for landing_page in landing_pages["elements"]:
            thread = threading.Thread(target=processar_landing_page, args=(landing_page, dados))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join()
        threads = []

    # Criar DataFrame
    df = pd.DataFrame(dados)

    # Salvar como Excel
    df.to_excel("landing_page_data_9.xlsx", index=False)
    print("Dados salvos em landing_page_data_threads.xlsx")

def main():
    microsite_id = 9
    processar_landing_pages_threads(microsite_id)

if __name__ == "__main__":
    main()

Processando landing page FL_20190321_ORM_Ohio_Google_Lunch_LP1...
Processando landing page FL_20190321_ORM_Ohio_Google_Lunch_LP2...
Processando landing page FL_20150727_mainContentOnly...
Processando landing page FL_2017_Welcome_LP2_OLD...
Processando landing page MOPS-CORP-PathFactory-Submits...
Processando landing page FL_2017PhoneIntakeForm...
Processando landing page MOPS-2016-TRTA-Testimonial-TY...
Processando landing page Default Integrated Webinar TY...
Processando landing page FL_2016_ORM_Event_Form_TY2...
Processando landing page FL_2016_ORM_Event_Form_TY3...
Processando landing page FL_2016_ORM_Event_Form_TY4...
Processando landing page FL_2016_ORM_Event_Form_TY5...
Processando landing page FL_2017_Insight_Report_HTTPS...
Processando landing page FL_2017_NPS_LP1...
Processando landing page FL_2017_NPS_LP1_Passive...
Processando landing page FL_2017_NPS_LP2...
Processando landing page FL_2017_ORM_Event_Form_TY6...
Processando landing page FL_2017_ORM_Event_Form_TY7...
Processa

Exception in thread Thread-1803 (processar_landing_page):
Traceback (most recent call last):
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\6125974\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\6125974\AppData\Local\Temp\ipykernel_24668\1386434429.py", line 57, in processar_landing_page
KeyError: 'constantValue'


Processando landing page C_OLI_SEM_9034130_ACCCoBrandPracticalLawJournalJan_20210216...
Processando landing page Tools - Canvas Name Generator_TEST...
Processando landing page TR Cross Referral Form TY Pg...
Processando landing page Tax Engine Webinar_Redirect_20210226...
Processando landing page TR Cross Referral Form...
Processando landing page C_WEB_9034965_DEIWebinar_20210325...
Processando landing page C_WEB_9034965_DEIWebinar_20210325 TY...
Processando landing page CORP_IDT_9034460_SAPLunchandLearn_03032021...
Processando landing page CORP_IDT_9034460_SAPLunchandLearn_03032021-TY...
Processando landing page CORP_PanTax_9035311_TEILunchandLearn_03112021...
Processando landing page CORP_OS_SAPInsiderPostEventSurvey_Attended-TY...
Processando landing page C_WEB_9035361_SCLDReflectionsWebinar_US_em1_WEBI_20210413 TY...
Processando landing page CORP_GT_ICPA_LunchandLearn-LP...
Processando landing page CORP_GT_ICPA_LunchandLearn-TY...
Processando landing page C_OLI_SEM_9026435_DaretoCo

Exception in thread Thread-2350 (processar_landing_page):
Traceback (most recent call last):
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\6125974\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\6125974\AppData\Local\Temp\ipykernel_24668\1386434429.py", line 57, in processar_landing_page
KeyError: 'constantValue'
Exception in thread Thread-2444 (processar_landing_page):
Traceback (most recent call last):
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\6125974\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_clo

Processando landing page C_LT_WEB_9072199_LDOIndexWebinar_US_20230119...
Processando landing page C_WEB_9066551_ContractIntelligence_US_20220915_TY...
Processando landing page C_LT_WEB_9069282_LTAWebinar_US_Tracker2022_G2_ABM...
Processando landing page C_LT_WEB_9073316_TrackerOpenData_US_20230201_TY...
Processando landing page Document Intelligence Form QA...
Processando landing page C_LT_WEB_9074742_LegalOpsGCReutersRB_US_20230406...
Processando landing page AEM-CORP-OSR-Forrester-webinar-2023-040423-TY...
Processando landing page C_OS_NURT_9071078_SSCForresterReport_AEM_ANZ_20230320_TY...
Processando landing page C_OS_NURT_9071078_SSCForresterReport_AEM_ANZ_20230320...
Processando landing page C_CLR_WEB_9073581_AdverseMedia_US_20230214_redirect...
Processando landing page AEM-CORP-IDT-Webinar-280323_LP_Japan...
Processando landing page AEM-CORP-IDT-Webinar-280323_LP_Japan-TY...
Processando landing page C_Wb_IDT_EInvoicing_0323_LP_redirect...
Processando landing page C_IDT_SEM_907605

Exception in thread Thread-2833 (processar_landing_page):
Traceback (most recent call last):
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-2835 (processar_landing_page):
Traceback (most recent call last):
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\6125974\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self.run()
  File "C:\Users\6125974\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    _threading_Thread_run(self)
  File "c:\Users\6125974\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs

Dados salvos em landing_page_data_threads.xlsx


: 

In [ ]:
import os
import requests
import pandas as pd
import threading

def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return (username, password)

def obter_landing_pages(microsite_id, page=1):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/assets/landingPages?search=micrositeId={microsite_id}&page={page}"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()  

    return response.json()

def obter_landing_page_detalhes(landing_page_id):
    url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/landingPage/{landing_page_id}"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()
    return response.json()

def obter_dados_landing_page(landing_page_id):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/data/report/5?assetId={landing_page_id}&startAt=1715620884"
    response = requests.get(url, auth=obter_autenticacao())
    response.raise_for_status()
    return response.json()

def processar_landing_page(landing_page, dados):
    landing_page_id = landing_page["id"]
    landing_page_name = landing_page["name"]

    print(f"Processando landing page {landing_page_name}...")

    # Obter detalhes da landing page para acessar os forms
    landing_page_detalhes = obter_landing_page_detalhes(landing_page_id)
    forms = landing_page_detalhes.get("forms", [])

    # Consultar dados da landing page
    dados_landing_page = obter_dados_landing_page(landing_page_id)

    # Calcular o retorno
    total = dados_landing_page["total"]
    retorno = "Y" if total > 0 else "N"

    if forms:
        for form in forms:
            form_name = form["name"]
            form_id = form["id"]
            redirect_name = ""

            # Procurar por FormStepRedirectToWebPage em processingSteps
            for step in form.get("processingSteps", []):
                if step["type"] == "FormStepRedirectToWebPage":
                    if "pageUrl" in step:
                        redirect_name = step["pageUrl"]["constantValue"]
                    elif "landingPageId" in step:
                        redirect_id = step["landingPageId"]["constantValue"]
                        # Obter detalhes do landing page de redirecionamento
                        redirect_details = obter_landing_page_detalhes(redirect_id)
                        redirect_name = redirect_details["name"]
                    else:
                        redirect_name = ""
                    break

            dados.append({
                "landingpage name": landing_page_name,
                "landingPage ID": landing_page_id,
                "Form Name": form_name,
                "Form ID": form_id,
                "Redirect Name": redirect_name,
                "Return": retorno  # Adicionando a coluna Return
            })
    else:
        # Se não houver forms, adiciona os campos em branco
        dados.append({
            "landingpage name": landing_page_name,
            "landingPage ID": landing_page_id,
            "Form Name": "",
            "Form ID": "",
            "Redirect Name": "",
            "Return": retorno  # Adicionando a coluna Return
        })

def processar_landing_pages_threads(microsite_id):
    # Obter a primeira página para descobrir o total e o pageSize
    landing_pages = obter_landing_pages(microsite_id)
    total = landing_pages["total"]
    pageSize = landing_pages["pageSize"]
    num_pages = (total // pageSize) + (1 if total % pageSize else 0)

    dados = []
    threads = []
    
    for page in range(1, num_pages + 1):
        landing_pages = obter_landing_pages(microsite_id, page)
        for landing_page in landing_pages["elements"]:
            thread = threading.Thread(target=processar_landing_page, args=(landing_page, dados))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join()
        threads = []

    # Criar DataFrame
    df = pd.DataFrame(dados)

    # Salvar como Excel
    df.to_excel("landing_page_data_9.xlsx", index=False)
    print("Dados salvos em landing_page_data_threads.xlsx")

def main():
    microsite_id = 9
    processar_landing_pages_threads(microsite_id)

if __name__ == "__main__":
    main()